In [86]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
%matplotlib inline
register_matplotlib_converters()


filename = '../data/demo_2020_0505.sqlite'
sql = '''SELECT * FROM inverter_minutely'''
with sqlite3.connect(filename) as con:
    df = pd.read_sql(sql, con=con)
    
    
df.LoggedDatetime = pd.to_datetime(df.LoggedDatetime) # convert string to pandas datetime object
df['minute'] = df.LoggedDatetime.dt.strftime('%Y-%m-%d %H:%M:00') # repalce seconds with 0

select_columns = ['DeviceID', 'minute', 'ACOutputPower']
df = df[select_columns]

df.set_index(['minute'], inplace=True)
df.head()

,DeviceID,ACOutputPower
minute,,
2020-05-05 05:12:00,1,0.0
2020-05-05 05:12:00,2,0.0
2020-05-05 05:12:00,3,0.0
2020-05-05 05:12:00,4,0.0
2020-05-05 05:12:00,5,0.0


In [87]:
def get_dfs(df):
    result = []
    for id in (1,2,3,4,5,18,19):
        condition = (df['DeviceID'] == id)
        invdf = df[condition]
        invdf = invdf[['ACOutputPower']]
        
        invdfh = invdf.groupby('minute').mean()
        invdfh = invdfh.rename(columns={'ACOutputPower': 'power-{}'.format(id)})
        result.append(invdfh)
    df = pd.concat(result, axis=1)
    return df

df = get_dfs(df)
df = df.dropna()
print(df.to_string())


                     power-1  power-2  power-3  power-4  power-5  power-18  power-19
minute                                                                              
2020-05-05 05:16:00     0.00     0.00     0.00     0.00     0.00      0.00      0.00
2020-05-05 05:17:00     0.00     0.00     0.00     0.00     0.00      0.00      0.00
2020-05-05 05:18:00     0.00     0.00     0.00     0.00     0.00      0.00      0.00
2020-05-05 05:19:00     0.00     0.00     0.00     0.00     0.00      0.00      0.00
2020-05-05 05:20:00     0.00     0.00     0.00     0.00     0.00      0.00      0.00
2020-05-05 05:21:00     0.00     0.00     0.00     0.00     0.00      0.00      0.00
2020-05-05 05:22:00     0.00     0.00     0.00     0.00     0.00      0.00      0.00
2020-05-05 05:23:00     0.00     0.00     0.00     0.00     0.00      0.00      0.00
2020-05-05 05:24:00     0.00     0.00     0.00     0.00     0.00      0.00      0.00
2020-05-05 05:25:00     0.00     0.00     0.00     0.00     0.00 

df.index = pd.DatetimeIndex(df.index)
df.index
df2 = df.resample('1H').mean()
df2

In [89]:
df3 = df2.corr()
df3

,power-1,power-2,power-3,power-4,power-5,power-18,power-19
power-1,1.000000,0.994042,0.999902,0.999367,0.999297,0.966701,0.908073
power-2,0.994042,1.000000,0.993112,0.989664,0.989395,0.963630,0.922244
power-3,0.999902,0.993112,1.000000,0.999625,0.999532,0.966105,0.906826
power-4,0.999367,0.989664,0.999625,1.000000,0.999982,0.964780,0.901176
power-5,0.999297,0.989395,0.999532,0.999982,1.000000,0.964274,0.900337
power-18,0.966701,0.963630,0.966105,0.964780,0.964274,1.000000,0.957855
power-19,0.908073,0.922244,0.906826,0.901176,0.900337,0.957855,1.000000


In [90]:
df3.mean().sort_values()

power-19    0.928073
power-18    0.969049
power-2     0.978870
power-5     0.978974
power-4     0.979228
power-3     0.980729
power-1     0.981054
dtype: float64